In [1]:
import pandas as pd
import json
import math
import sqlite3
import datetime
import html_to_json
year = 2025
pd.set_option('display.max_columns', None)
cnx = sqlite3.connect('results.db')

## Janowiec Wielkopolski

In [2]:
results = []
with open('janowiec-wielkopolski.txt') as f:
    for line in f.readlines():
        parts = line.split('\t')
        results.append([parts[5] + ' ' + parts[7], parts[9], (parts[15] or parts[14])[:1], (parts[19] or parts[18])[:3], parts[11] or None, int(next(p for p in parts if len(p.strip()) == 4 and p[:2] in {'19', '20'})), parts[-1].strip() or parts[-2].strip()])

janowiec = pd.DataFrame(results, columns=[
    'name',
    'city',
    'gender',
    'category',
    'team',
    'year_of_birth',
    'result',
])
assert not any(janowiec['name'].duplicated())
# sorted(janowiec[janowiec['team'].notnull()]['team'].unique())
janowiec.loc[janowiec['team'] == 'TRACK GANG', 'team'] = 'TRACK GANG TORUŃ'
janowiec.loc[janowiec['team'] == 'WATAHA ŁĄCZY NAS BIEGANIE', 'team'] = 'WATAHA - ŁĄCZY NAS BIEGANIE'
janowiec.loc[janowiec['team'] == 'EKO- OPAŁ PŁONNE', 'team'] = 'EKO-OPAŁ PŁONNE'
janowiec.loc[janowiec['name'] == 'KRIGER- PRZYBYLSKA ANNA', 'name'] = 'KRIGER-PRZYBYLSKA ANNA'
janowiec.loc[janowiec['name'] == 'WISNIEWSKI DAMIAN', 'name'] = 'WIŚNIEWSKI DAMIAN'

janowiec

,name,city,gender,category,team,year_of_birth,result
0,ABRAMOWSKI DAMIAN,ŻNIN,M,M30,TT SODA,1991,00:37:45
1,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,K40,None,1985,00:47:33
2,ADAMSKA ANNA,KŁECKO,K,K40,LECHITA BIEGA KŁECKO,1985,00:57:09
3,ADAMSKI ADAM,ŻNIN,M,M30,None,1986,00:47:48
4,ANGIEL WŁADYSŁAW,BYDGOSZCZ,M,M80,None,1937,01:40:32
...,...,...,...,...,...,...,...
267,ZMUDZIŃSKA JOANNA,PIECHCIN,K,K40,None,1983,00:45:37
268,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,M50,BYDGOSKIE ŻÓŁWIE,1973,00:50:43
269,ŁASIŃSKI DARIUSZ,KOWALEWO,M,M30,None,1991,00:44:20
270,ŻUCHOWSKI JÓZEF,PŁONNE,M,M50,EKO-OPAŁ PŁONNE,1971,00:41:23


## Fabianki

In [3]:
fabianki = pd.read_csv('fabianki.txt', sep='\t', names=['c0', 'c1', 'name', 'gender', 'c2', 'c3', 'category', 'c5', 'team', 'c6', 'c7', 'c8', 'c9', 'result', 'c11'])
fabianki = fabianki[['name', 'gender', 'category', 'team', 'result']]
fabianki['name'] = fabianki['name'].str.upper()
fabianki['team'] = fabianki['team'].str.upper()
fabianki = fabianki[fabianki['result'].notnull()]
fabianki['category'] = fabianki['category'].str.slice(0, 3)

fabianki.loc[fabianki['team'] == 'PAŁUKI RUNNIG TEAM', 'team'] = 'PAŁUKI RUNNING TEAM'
fabianki.loc[fabianki['team'] == 'T G SOKÓŁ GNIEWKOWO', 'team'] = 'TG SOKÓŁ GNIEWKOWO'
fabianki.loc[fabianki['team'] == 'T.G.SOKÓŁ GNIEWKOWO', 'team'] = 'TG SOKÓŁ GNIEWKOWO'
fabianki.loc[fabianki['team'] == 'WKB MARATONCZYK WLOCLAWEK', 'team'] = 'WKB MARATOŃCZYK WŁOCŁAWEK'
fabianki.loc[fabianki['team'] == 'WKB MARATONCZYK WŁOCŁAWEK', 'team'] = 'WKB MARATOŃCZYK WŁOCŁAWEK'
fabianki.loc[fabianki['name'] == 'BLASZCZAK STANISLAW', 'name'] = 'BŁASZCZAK STANISŁAW'
fabianki.loc[fabianki['name'] == 'WŁADYSŁAW ANGIEL', 'name'] = 'ANGIEL WŁADYSŁAW'

any(fabianki['name'].duplicated())

fabianki

,name,gender,category,team,result
0,KLEDZIK ZBIGNIEW,M,M30,PAŁUKI RUNNING TEAM,00:32:21
1,TOMASZEWSKI MIŁOSZ,M,M30,GKS FABIANKI,00:33:05
2,TOMASZEWSKI MACIEJ,M,M30,TASMAN TEAM,00:33:46
3,KUNECKI DARIUSZ,M,M40,KUNA TEAM WŁOCŁAWEK,00:34:42
4,SZTOLCMAN PIOTR,M,M30,GKS FABIANKI,00:34:55
...,...,...,...,...,...
161,GLAZIK RENATA,K,K50,FABIANKI,01:18:26
162,WITKOWSKA EDYTA,K,K50,FABIANKI,01:18:32
163,WITKOWSKA JULIA,K,K16,FABIANKI,01:18:33
164,ANDELBRAT BEATA,K,K50,GRUDZIADZ,01:18:33


## Radziejów

In [4]:
results = []
with open('radziejow.json') as f:
    data = json.load(f)
    for row in data['data']:
        # for idx, f in enumerate(row):
        #     print(idx, f)
        # xxx
        if len(row) != 19:
            continue
        if row[14] == 'DNS':
            continue
        results.append([
            row[3].upper(),
            row[8].upper(),
            row[11][:1],
            row[7].upper() if row[7] else None,
            row[4],
            row[12].zfill(8),
        ])

radziejow = pd.DataFrame(results, columns=[
    'name',
    'city',
    'gender',
    'team',
    'year_of_birth',
    'result_radziejow',
])

fixes = [
    ('team', 'SOLTYS', 'SOŁTYS SUCHARZEWO'),
    ('team', 'WATAHA ŁĄCZY NAS BIEGANIE', 'WATAHA - ŁĄCZY NAS BIEGANIE'),
    ('city', 'WŁOCŁAWK', 'WŁOCŁAWEK'),
    ('name', 'GOLEBIEWSKI MICHAL', 'GOŁĘBIEWSKI MICHAŁ'),
    ('name', 'HUSSEIN ABDALLAH', 'HUSSEIN ABDALLA'),
    ('name', 'DLONIAK EDMUND', 'DŁONIAK EDMUND'),
    ('name', 'ZIÓŁKOWSKI MIECZYSLAW', 'ZIÓŁKOWSKI MIECZYSŁAW'),
]
for field, from_, to_ in fixes:
    radziejow.loc[radziejow[field] == from_, field] = to_

radziejow

,name,city,gender,team,year_of_birth,result_radziejow
0,RATAJCZYK PRZEMYSŁAW,BYDGOSZCZ,M,TKKF KOLEJARZ BYDGOSZCZ,1993,00:32:48
1,KLEDZIK ZBIGNIEW,KIERZKOWO,M,PAŁUKI RUNNING TEAM,1991,00:32:57
2,TOMASZEWSKI MACIEJ,TORUŃ,M,TASMAN TEAM,1988,00:33:04
3,JUREK ŁUKASZ,JANIKOWO,M,STOWARZYSZENIE SIŁA,1989,00:33:57
4,STANISZEWSKI SŁAWOMIR,ŻNIN,M,PAŁUKI RUNNING TEAM,1989,00:35:23
...,...,...,...,...,...,...
159,BONIECKA TERESA,RYPIN,K,RKS LECH RYPIN,1974,01:11:07
160,KRZYŻANIAK BOGDAN,ZAKRZEWO,M,None,1961,01:11:39
161,BŁASZCZAK STANISŁAW,SKEPE,M,WKB MARATOŃCZYK WŁOCŁAWEK,1946,01:12:43
162,LENC JANUSZ,BYDGOSZCZ,M,TKKF KOLEJARZ BYDGOSZCZ,1953,01:23:40


## Lisewo

In [5]:
lisewo = pd.read_csv('lisewo.csv')
lisewo['name'] = (lisewo['Last_Name'] + ' ' + lisewo['First_Name']).str.upper()
lisewo['city'] = lisewo['City'].str.upper()
lisewo['team'] = lisewo['Club'].str.upper()
lisewo['year_of_birth'] = lisewo['Birth_Year']
lisewo['result_lisewo'] = '0' + lisewo['Net_Time']
lisewo['gender'] = lisewo['Category'].str[:1]
lisewo = lisewo[['name', 'city', 'team', 'year_of_birth', 'result_lisewo', 'gender']]


fixes = [
    ('team', 'SOŁTYS', 'SOŁTYS SUCHARZEWO'),
    ('team', 'WATAHA ŁĄCZY NAS BIEGANIE', 'WATAHA - ŁĄCZY NAS BIEGANIE'),
    ('team', 'RKS LECH RPIN', 'RKS LECH RYPIN'),
    ('team', 'TG SOKÓŁ', 'TG SOKÓŁ GNIEWKOWO'),
    ('team', 'T.G.SOKÓŁ GNIEWKOWO', 'TG SOKÓŁ GNIEWKOWO'),
    ('team', 'ZAKOLATEAM', '#ZAKOLATEAM'),
    ('team', 'KUNA TEAM', 'KUNA TEAM WŁOCŁAWEK'),
    ('team', 'WATAHA ŁĄCZY NAS BIEGANIE', 'WATAHA - ŁĄCZY NAS BIEGANIE'),
    ('team', 'T G SOKÓŁ GNIEWKOWO', 'TG SOKÓŁ GNIEWKOWO'),
    ('city', 'WLOCLAWEK', 'WŁOCŁAWEK'),
    ('city', 'WŁOCŁAWK', 'WŁOCŁAWEK'),
    ('city', 'TORUN', 'TORUŃ'),
    ('name', 'ABDALLA HUSSEIN', 'HUSSEIN ABDALLA'),
    ('name', 'ŻUCHOWSKI JOZEF', 'ŻUCHOWSKI JÓZEF'),
    ('name', 'MATUSZEWSKA ELZBIETA', 'MATUSZEWSKA ELŻBIETA'),
]
for field, from_, to_ in fixes:
    lisewo.loc[lisewo[field] == from_, field] = to_

lisewo

,name,city,team,year_of_birth,result_lisewo,gender
0,TOMASZEWSKI MACIEJ,TORUŃ,TASMAN TEAM,1988,00:33:15,M
1,PIASECKI KRZYSZTOF,WŁOCŁAWEK,KUNA TEAM WŁOCŁAWEK,1988,00:35:50,M
2,KUNECKI DARIUSZ,WŁOCŁAWEK,KUNA TEAM WŁOCŁAWEK,1984,00:36:02,M
3,SIKUT MICHAŁ,TORUŃ,ZALEWSKI PERFORMANCE ACADEMY,1995,00:36:44,M
4,KWASIBORSKI PATRYK,ROŚCISZEWO,GKS FABIANKI,1996,00:36:53,M
...,...,...,...,...,...,...
164,BONIECKA TERESA,RADOMIN,RKS LECH RYPIN,1974,01:11:31,K
165,BŁASZCZAK STANISŁAW,SKĘPE,WKB MARATOŃCZYK WŁOCŁAWEK,1946,01:14:28,M
166,BIAŁOBRZESKA EWA MARLENA,RUSINOWO,RKS LECH RYPIN,1961,01:19:55,K
167,GERKA KRZYSZTOF,BRZUZE,RKS LECH RYPIN,1957,01:19:51,M


## Piotrków

In [6]:
piotrkow = pd.read_csv('piotrkow.txt', sep='\t', names=['c0', 'c1', 'name', 'category', 'c2', 'team', 'result_piotrkow'])
piotrkow = piotrkow[['name', 'category', 'team', 'result_piotrkow']]
piotrkow['gender'] = piotrkow['category'].str.slice(0, 1)
piotrkow['name'] = piotrkow['name'].str.upper()
piotrkow['team'] = piotrkow['team'].str.upper()

fixes = [
    ('team', 'GKS FABIANKI.', 'GKS FABIANKI'),
    ('team', 'T G SOKÓŁ GNIEWKOWO', 'TG SOKÓŁ GNIEWKOWO'),
    ('team', 'GKS FABIANKI / UMT TORUŃ', 'GKS FABIANKI'),
    ('team', 'PAŁUKI RUNING TEAM', 'PAŁUKI RUNNING TEAM'),
    ('team', 'KS ZRYW', 'KS ZRYW RADZIEJÓW'),
    ('team', 'ZAKOLATEAM', '#ZAKOLATEAM'),
    ('team', 'PARKRUN', 'PARKRUN TORUŃ'),
    ('team', 'ZRYW RADZIEJÓW', 'KS ZRYW RADZIEJÓW'),
    ('name', 'KUDLA AGNIESZKA', 'KUDŁA AGNIESZKA'),
    ('name', 'BŁASZCZAK STANISLAW', 'BŁASZCZAK STANISŁAW'),
]
for field, from_, to_ in fixes:
    piotrkow.loc[piotrkow[field] == from_, field] = to_

piotrkow

,name,category,team,result_piotrkow,gender
0,KLEDZIK ZBIGNIEW,M30,PAŁUKI RUNNING TEAM,00:32:40,M
1,TOMASZEWSKI MACIEJ,M30,TASMAN TEAM,00:32:54,M
2,JANKOWIAK DAREK,M30,CARSEKT,00:34:17,M
3,PIASECKI KRZYSZTOF,M30,KUNA TEAM WŁOCŁAWEK,00:35:07,M
4,GRUDZIŃSKI MACIEJ,M30,WKB MARATOŃCZYK WŁOCŁAWEK,00:35:31,M
...,...,...,...,...,...
218,SPIBIDA KLAUDIA,K30,EKONSTAL,01:24:00,K
219,KURDUPSKA ILONA,K30,BYTOŃ,01:24:01,K
220,LENC JANUSZ,M70,TKKF KOLEJARZ BYDGOSZCZ,01:26:58,M
221,GERKA KRZYSZTOF,M60,RKS LECH RYPIN,01:27:00,M


# Golub

In [7]:
## curl 'https://live.sts-timing.pl/bows2025/api.php?action=load-more-results&i=100&d=1&p=false&filtr=false' >> git/parkrun-torun/wyniki-gp/golub.json

html = '<table>'
with open('golub.json') as f:
    for line in f.readlines():
        html += json.loads(line)['html']

html += '</table>'
table = html_to_json.convert_tables(html)[0]
golub = pd.DataFrame(table, columns=[
    'place',
    'start_nr',
    'name',
    'country',
    'city',
    'team',
    'gender',
    '_c0',
    'category',
    '_c1',
    'gun_time',
    'result_golub',
    '_c2',
])
golub = golub[['name', 'city', 'team', 'gender', 'category', 'result_golub']]
golub['name'] = golub['name'].str.upper()
golub['city'] = golub['city'].str.upper()
golub['team'] = golub['team'].str.upper().replace('', None)
golub['result_golub'] = golub['result_golub'].str.zfill(8).replace('^000', '00:', regex=True)

fixes = [
    ('team', 'PARKRUN', 'PARKRUN TORUŃ'),
    ('team', 'SOŁTYS', 'SOŁTYS SUCHARZEWO'),
    ('team', 'ZAKOLATEAM', '#ZAKOLATEAM'),
    ('team', 'PARKRUN', 'PARKRUN TORUŃ'),
    ('team', 'T.G.SOKÓŁ', 'TG SOKÓŁ GNIEWKOWO'),
    ('team', 'T.G..SOKÓŁ GNIEWKOWO', 'TG SOKÓŁ GNIEWKOWO'),
    ('city', 'TORUN', 'TORUŃ'),
]
for field, from_, to_ in fixes:
    golub.loc[golub[field] == from_, field] = to_
    
golub

,name,city,team,gender,category,result_golub
0,PRZYBYŁA ADRIAN,IŁAWA,RUN GOKU TEAM,M,M30,00:32:08
1,TOMASZEWSKI MIŁOSZ,TORUŃ,GKS FABIANKI,M,M30,00:32:20
2,KWASIBORSKI PATRYK,NOWE ROŚCISZEWO,GKS FABIANKI,M,M16,00:35:30
3,SZYMAŃSKI MARCIN,BOBROWNIKI,WKB MARATOŃCZYK WŁOCŁAWEK,M,M40,00:35:48
4,KUNECKI DARIUSZ,WŁOCŁAWEK,KUNA TEAM WŁOCŁAWEK,M,M40,00:37:17
...,...,...,...,...,...,...
174,MAZUR GABRIELA,GDYNIA,None,K,K50,01:10:07
175,BONIECKA TERESA,RYPIN,RKS LECH RYPIN,K,K50,01:11:16
176,MIDDELVELD LAURA,DOBIEGNIEW,None,K,K16,01:13:17
177,ŁUKASZEWSKA JAGODA,GOLUB-DOBRZYŃ,None,K,K16,01:13:17


# Rypin

In [8]:
rypin = pd.read_csv('rypin.csv')
rypin['name'] = rypin['Nazwisko'] + ' ' + rypin['Imię']
rypin['team'] = rypin['Klub']
rypin['city'] = rypin['Miejscowość']
rypin['gender'] = rypin['Kat'].str.slice(0, 1)
rypin['category'] = rypin['Kat'].str.slice(0, 1) + rypin['Kat'].str.slice(2, 4) 
rypin['result_rypin'] = rypin['Meta'].str.slice(0, 8)
rypin = rypin[['name', 'team', 'city', 'gender', 'category', 'result_rypin']]

fixes = [
    ('team', 'SOŁTYS', 'SOŁTYS SUCHARZEWO'),
    ('team', 'T.G.SOKÓŁ GNIEWKOWO', 'TG SOKÓŁ GNIEWKOWO'),
    ('city', 'TORUN', 'TORUŃ'),
    ('city', 'WLOCLAWEK', 'WŁOCŁAWEK'),
    ('city', 'WŁOCŁAWK', 'WŁOCŁAWEK'),
    ('name', 'KRIGER–PRZYBYLSKA ANNA', 'KRIGER-PRZYBYLSKA ANNA'),
    ('name', 'ROK-SZPALA SYLWIA ROK-SZPALA', 'ROK-SZPALA SYLWIA'),
    ('name', 'MECHLIŃSKI BARTŁOMIEJ MECHLIŃSKI', 'MECHLIŃSKI BARTŁOMIEJ'),
    ('name', 'ŻUCHOWSKI JÓZEF ŻUCHOWSKI', 'ŻUCHOWSKI JÓZEF'),
    ('name', 'SZTENDEREWICZ RENATA SZTENDEREWICZ', 'SZTENDEREWICZ RENATA'),
    ('name', 'MARKOWSKA EWA MARKOWSKA', 'EWA MARKOWSKA'),
]
for field, from_, to_ in fixes:
    rypin.loc[rypin[field] == from_, field] = to_

rypin

,name,team,city,gender,category,result_rypin
0,TOMASZEWSKI MIŁOSZ,GKS FABIANKI,TORUŃ,M,M30,00:31:24
1,CZARNECKI PRZEMYSŁAW,CHOJNICE BIEGAJĄ,CHOJNICE,M,M30,00:33:55
2,TOMASZEWSKI MACIEJ,GKS FABIANKI,TORUŃ,M,M30,00:34:08
3,PIASECKI KRZYSZTOF,KUNA TEAM WŁOCŁAWEK,WŁOCŁAWEK,M,M30,00:35:21
4,KUNECKI DARIUSZ,KUNA TEAM WŁOCŁAWEK,WŁOCŁAWEK,M,M40,00:35:35
...,...,...,...,...,...,...
175,NOWACZYK DOROTA,NaN,TORUŃ,K,K40,01:13:10
176,KOZŁOWSKA DANUTA,RKS LECH RYPIN,SIERPC,K,K70,01:18:30
177,BŁASZCZAK STANISŁAW,WKB MARATOŃCZYK WŁOCŁAWEK,SKĘPE,M,M70,01:20:37
178,BIAŁOBŁOCKA EWA,RKS LECH RYPIN,RYPIN,K,K60,01:22:38


## Merge

In [9]:
after_2 = janowiec.merge(fabianki, on=['name', 'gender'], how='outer', suffixes=('_janowiec', '_fabianki'))

In [10]:
after_2[after_2['team_janowiec'].notnull() & after_2['team_fabianki'].notnull() & (after_2['team_janowiec'] != after_2['team_fabianki'])]

,name,city,gender,category_janowiec,team_janowiec,year_of_birth,result_janowiec,category_fabianki,team_fabianki,result_fabianki
72,GRABARSKA MAGDALENA,BRZOZÓWKA,K,K30,TRACK GANG TORUŃ,1986.0,00:47:21,K30,BRZOZÓWKA,00:45:24
118,KARPIŃSKI PAWEŁ,GOLUB-DOBRZYŃ,M,M20,ZAKOLATEAMFAMILIA,1996.0,00:39:14,M16,#ZAKOLATEAM,00:38:35
320,TOMASZEWSKI WOJCIECH,TORUŃ,M,M60,START TEAM WĄBRZEŹNO,1964.0,00:41:14,M60,TASMAN TEAM,00:39:40


In [11]:
after_2[after_2['category_janowiec'].notnull() & after_2['category_fabianki'].notnull() & (after_2['category_janowiec'] != after_2['category_fabianki'])]

,name,city,gender,category_janowiec,team_janowiec,year_of_birth,result_janowiec,category_fabianki,team_fabianki,result_fabianki
118,KARPIŃSKI PAWEŁ,GOLUB-DOBRZYŃ,M,M20,ZAKOLATEAMFAMILIA,1996.0,00:39:14,M16,#ZAKOLATEAM,00:38:35
234,PIETRZAK MICHAŁ,WŁOCŁAWEK,M,M20,KUNA TEAM WŁOCŁAWEK,2005.0,00:36:42,M16,KUNA TEAM WŁOCŁAWEK,00:36:28


In [12]:
after_2['team'] = after_2['team_fabianki'].combine_first(after_2['team_janowiec'])
after_2['category'] = after_2['category_fabianki'].combine_first(after_2['category_janowiec'])
after_2 = after_2.drop(labels=['category_janowiec', 'team_janowiec', 'category_fabianki', 'team_fabianki'], axis=1)
after_2['year_of_birth'] = after_2['year_of_birth'].astype('Int64')

#### Radziejów

In [13]:
after_3 = after_2.merge(radziejow, on=['name', 'gender'], how='outer', suffixes=('_m', '_radziejow'))
after_3['year_of_birth_radziejow'] = after_3['year_of_birth_radziejow'].astype('Int64')
after_3

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow
0,ABRAMOWSKI DAMIAN,ŻNIN,M,1991,00:37:45,NaN,TT SODA,M30,NaN,NaN,<NA>,NaN
1,ADAMSKA ANNA,KŁECKO,K,1985,00:57:09,NaN,LECHITA BIEGA KŁECKO,K40,NaN,NaN,<NA>,NaN
2,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,1985,00:47:33,NaN,None,K40,NaN,NaN,<NA>,NaN
3,ADAMSKI ADAM,ŻNIN,M,1986,00:47:48,NaN,None,M30,NaN,NaN,<NA>,NaN
4,ANDELBRAT BEATA,NaN,K,<NA>,NaN,01:18:33,GRUDZIADZ,K50,NaN,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
443,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,1973,00:50:43,NaN,BYDGOSKIE ŻÓŁWIE,M50,NaN,NaN,<NA>,NaN
444,ŻABIŃSKI RADOSŁAW,NaN,M,<NA>,NaN,NaN,NaN,NaN,WŁOCŁAWEK,WKB MARATOŃCZYK WŁOCŁAWEK,1986,00:51:21
445,ŻUCHOWSKI BARTOSZ,NaN,M,<NA>,NaN,00:37:34,PŁONNE,M16,PŁONNE,None,1996,00:38:12
446,ŻUCHOWSKI JÓZEF,PŁONNE,M,1971,00:41:23,00:40:08,EKO-OPAŁ PŁONNE,M50,NaN,NaN,<NA>,NaN


In [14]:
after_3[after_3['team_m'].notnull() & after_3['team_radziejow'].notnull() & (after_3['team_m'] != after_3['team_radziejow'])]

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow
233,MECHLIŃSKI BARTŁOMIEJ,NaN,M,<NA>,NaN,00:37:00,TORUŃ,M40,TORUŃ,BANANOWY WOJOWNIK,1979,00:37:35


In [15]:
after_3[after_3['year_of_birth_radziejow'].notnull() & after_3['year_of_birth_m'].notnull() & (after_3['year_of_birth_radziejow'] != after_3['year_of_birth_m'])]

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow


In [16]:
after_3[after_3['city_radziejow'].notnull() & after_3['city_m'].notnull() & (after_3['city_radziejow'] != after_3['city_m'])]

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow
89,GRUDZIŃSKI MACIEJ,ŁÓDŹ,M,1986,00:37:13,NaN,WKB MARATOŃCZYK WŁOCŁAWEK,M30,WŁOCŁAWEK,WKB MARATOŃCZYK WŁOCŁAWEK,1986,00:36:15
104,HUSSEIN ABDALLA,ODOLION,M,1987,00:44:03,NaN,PARKRUN TORUŃ,M30,KONRADOWO,PARKRUN TORUŃ,1987,00:41:50
105,HUSSEIN ANNA,ODOLION,K,1981,00:51:12,NaN,PARKRUN TORUŃ,K40,KONRADOWO,PARKRUN TORUŃ,1981,00:49:09


In [17]:
after_3['team'] = after_3['team_radziejow'].combine_first(after_3['team_m'])
after_3['year_of_birth'] = after_3['year_of_birth_radziejow'].combine_first(after_3['year_of_birth_m'])
after_3['city'] = after_3['city_radziejow'].combine_first(after_3['city_m'])
after_3 = after_3.drop(labels=['team_radziejow', 'team_m', 'year_of_birth_radziejow', 'year_of_birth_m', 'city_radziejow', 'city_m'], axis=1)

### Lisewo

In [18]:
after_4 = after_3.merge(lisewo, on=['name', 'gender'], how='outer', suffixes=('_m', '_lisewo'))
after_4['year_of_birth_lisewo'] = after_4['year_of_birth_lisewo'].astype('Int64')
after_4

,name,gender,result_janowiec,result_fabianki,category,result_radziejow,team_m,year_of_birth_m,city_m,city_lisewo,team_lisewo,year_of_birth_lisewo,result_lisewo
0,ABRAMOWSKI DAMIAN,M,00:37:45,NaN,M30,NaN,TT SODA,1991,ŻNIN,NaN,NaN,<NA>,NaN
1,ADAMSKA ANNA,K,00:57:09,NaN,K40,NaN,LECHITA BIEGA KŁECKO,1985,KŁECKO,NaN,NaN,<NA>,NaN
2,ADAMSKA IZABELA,K,00:47:33,NaN,K40,NaN,None,1985,JANOWIEC WIELKOPOLSKI,NaN,NaN,<NA>,NaN
3,ADAMSKI ADAM,M,00:47:48,NaN,M30,NaN,None,1986,ŻNIN,NaN,NaN,<NA>,NaN
4,ANDELBRAT BEATA,K,NaN,01:18:33,K50,NaN,GRUDZIADZ,<NA>,NaN,NaN,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,ŚWIĘCICKI ANDRZEJ,M,00:50:43,NaN,M50,NaN,BYDGOSKIE ŻÓŁWIE,1973,BYDGOSZCZ,NaN,NaN,<NA>,NaN
528,ŻABIŃSKI RADOSŁAW,M,NaN,NaN,NaN,00:51:21,WKB MARATOŃCZYK WŁOCŁAWEK,1986,WŁOCŁAWEK,NaN,NaN,<NA>,NaN
529,ŻUCHOWSKI BARTOSZ,M,NaN,00:37:34,M16,00:38:12,PŁONNE,1996,PŁONNE,NaN,NaN,<NA>,NaN
530,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,M50,NaN,EKO-OPAŁ PŁONNE,1971,PŁONNE,RADOMIN,NaN,1971,00:41:41


In [19]:
after_4[after_4['team_m'].notnull() & after_4['team_lisewo'].notnull() & (after_4['team_m'] != after_4['team_lisewo'])]

,name,gender,result_janowiec,result_fabianki,category,result_radziejow,team_m,year_of_birth_m,city_m,city_lisewo,team_lisewo,year_of_birth_lisewo,result_lisewo
112,GRABARSKA MAGDALENA,K,00:47:21,00:45:24,K30,NaN,BRZOZÓWKA,1986,BRZOZÓWKA,BRZOZÓWKA,TRACK GANG,1986,00:47:48
404,STAWARSKA ELŻBIETA,K,00:49:31,00:47:01,K40,00:48:28,TORUŃ,1977,TORUŃ,TORUŃ,GP,1977,00:49:52


In [20]:
after_4[after_4['year_of_birth_lisewo'].notnull() & after_4['year_of_birth_m'].notnull() & (after_4['year_of_birth_lisewo'] != after_4['year_of_birth_m'])]

,name,gender,result_janowiec,result_fabianki,category,result_radziejow,team_m,year_of_birth_m,city_m,city_lisewo,team_lisewo,year_of_birth_lisewo,result_lisewo


In [21]:
after_4[after_4['city_lisewo'].notnull() & after_4['city_m'].notnull() & (after_4['city_lisewo'] != after_4['city_m'])]

,name,gender,result_janowiec,result_fabianki,category,result_radziejow,team_m,year_of_birth_m,city_m,city_lisewo,team_lisewo,year_of_birth_lisewo,result_lisewo
28,BONIECKA TERESA,K,01:14:12,01:10:23,K50,01:11:07,RKS LECH RYPIN,1974,RYPIN,RADOMIN,RKS LECH RYPIN,1974,01:11:31
42,BŁASZCZAK STANISŁAW,M,NaN,01:12:55,M70,01:12:43,WKB MARATOŃCZYK WŁOCŁAWEK,1946,SKEPE,SKĘPE,WKB MARATOŃCZYK WŁOCŁAWEK,1946,01:14:28
60,CZERENKIEWICZ-PAŁUCKA JOANNA,K,00:50:02,00:49:15,K50,00:49:53,KUNA TEAM WŁOCŁAWEK,1975,WŁOCŁAWEK,BRZEŚĆ KUJ.,KUNA TEAM WŁOCŁAWEK,1975,00:51:07
61,CZERWIŃSKI KRZYSZTOF,M,NaN,00:37:51,M16,00:38:18,WKB MARATOŃCZYK WŁOCŁAWEK,1996,OSNO,CIECHOCINEK,WKB MARATOŃCZYK WŁOCŁAWEK,1996,00:39:37
93,DĘBSKI JAN,M,00:59:06,00:57:49,M70,00:59:44,SOŁTYS SUCHARZEWO,1952,SUCHARZEWO,DĄBROWA,SOŁTYS SUCHARZEWO,1952,01:00:24
104,GERKA KRZYSZTOF,M,NaN,01:09:56,M60,01:10:21,RKS LECH RYPIN,1957,RYPIN,BRZUZE,RKS LECH RYPIN,1957,01:19:51
210,KOTUŁA PAWEŁ,M,NaN,NaN,NaN,00:45:00,START TEAM WĄBRZEŹNO,1979,TRZCIANO,RYŃSK,START TEAM WĄBRZEŹNO,1979,00:40:28
244,KWASIBORSKI PATRYK,M,NaN,00:35:10,M16,00:36:17,GKS FABIANKI,1996,NOWE ROŚCISZEWO,ROŚCISZEWO,GKS FABIANKI,1996,00:36:53
310,OLSZEWSKI ADAM,M,NaN,00:50:25,M60,00:53:00,RKS LECH RYPIN,1963,BRZUZE,ROGOWO,NaN,1963,00:53:29
370,ROSIŃSKA ANNA,K,NaN,NaN,NaN,00:54:50,NaN,1992,SZEWCE 33,PIOTRKÓW KUJAWSKI,NaN,1992,00:59:15


In [22]:
after_4['team'] = after_4['team_lisewo'].combine_first(after_4['team_m'])
after_4['year_of_birth'] = after_4['year_of_birth_lisewo'].combine_first(after_4['year_of_birth_m'])
after_4['city'] = after_4['city_lisewo'].combine_first(after_4['city_m'])
after_4 = after_4.drop(labels=['team_lisewo', 'team_m', 'year_of_birth_lisewo', 'year_of_birth_m', 'city_lisewo', 'city_m'], axis=1)

### Piotrków

In [23]:
after_5 = after_4.merge(piotrkow, on=['name', 'gender'], how='outer', suffixes=('_m', '_piotrkow'))
after_5

,name,gender,result_janowiec,result_fabianki,category_m,result_radziejow,result_lisewo,team_m,year_of_birth,city,category_piotrkow,team_piotrkow,result_piotrkow
0,ABRAMOWSKI DAMIAN,M,00:37:45,NaN,M30,NaN,NaN,TT SODA,1991,ŻNIN,NaN,NaN,NaN
1,ADAMKIEWICZ SŁAWOMIR,M,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,M50,SGB TEAM,00:48:21
2,ADAMSKA ANNA,K,00:57:09,NaN,K40,NaN,NaN,LECHITA BIEGA KŁECKO,1985,KŁECKO,NaN,NaN,NaN
3,ADAMSKA IZABELA,K,00:47:33,NaN,K40,NaN,NaN,None,1985,JANOWIEC WIELKOPOLSKI,NaN,NaN,NaN
4,ADAMSKI ADAM,M,00:47:48,NaN,M30,NaN,NaN,None,1986,ŻNIN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,ŚWIĘCICKI ANDRZEJ,M,00:50:43,NaN,M50,NaN,NaN,BYDGOSKIE ŻÓŁWIE,1973,BYDGOSZCZ,NaN,NaN,NaN
619,ŻABIŃSKI RADOSŁAW,M,NaN,NaN,NaN,00:51:21,NaN,WKB MARATOŃCZYK WŁOCŁAWEK,1986,WŁOCŁAWEK,NaN,NaN,NaN
620,ŻUCHOWSKI BARTOSZ,M,NaN,00:37:34,M16,00:38:12,NaN,PŁONNE,1996,PŁONNE,NaN,NaN,NaN
621,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,M50,NaN,00:41:41,EKO-OPAŁ PŁONNE,1971,RADOMIN,NaN,NaN,NaN


In [24]:
after_5[after_5['team_m'].notnull() & after_5['team_piotrkow'].notnull() & (after_5['team_m'] != after_5['team_piotrkow'])]

,name,gender,result_janowiec,result_fabianki,category_m,result_radziejow,result_lisewo,team_m,year_of_birth,city,category_piotrkow,team_piotrkow,result_piotrkow
331,MECHLIŃSKI BARTŁOMIEJ,M,NaN,00:37:00,M40,00:37:35,NaN,BANANOWY WOJOWNIK,1979,TORUŃ,M40,TORUŃ,00:37:03
472,STAWARSKA ELŻBIETA,K,00:49:31,00:47:01,K40,00:48:28,00:49:52,GP,1977,TORUŃ,K40,TORUŃ,00:50:17
588,WĄŻ SŁAWOMIR,M,NaN,NaN,NaN,NaN,00:59:40,KLUB SPORTOWY PADALEC,1972,GRĘBOCIN,M50,WOLLT IHR DAS BETT IN FLAMMEN SEHEN,01:06:55


In [25]:
after_5['team'] = after_5['team_piotrkow'].combine_first(after_5['team_m'])
after_5['category'] = after_5['category_piotrkow'].combine_first(after_5['category_m'])
after_5 = after_5.drop(labels=['team_piotrkow', 'team_m', 'category_piotrkow', 'category_m'], axis=1)

# Golub

In [26]:
after_6 = after_5.merge(golub, on=['name', 'gender'], how='outer', suffixes=('_m', '_golub'))
after_6

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,city_m,result_piotrkow,team_m,category_m,city_golub,team_golub,category_golub,result_golub
0,ABRAMOWSKI DAMIAN,M,00:37:45,NaN,NaN,NaN,1991,ŻNIN,NaN,TT SODA,M30,NaN,NaN,NaN,NaN
1,ADAMKIEWICZ SŁAWOMIR,M,NaN,NaN,NaN,NaN,<NA>,NaN,00:48:21,SGB TEAM,M50,NaN,NaN,NaN,NaN
2,ADAMSKA ANNA,K,00:57:09,NaN,NaN,NaN,1985,KŁECKO,NaN,LECHITA BIEGA KŁECKO,K40,NaN,NaN,NaN,NaN
3,ADAMSKA IZABELA,K,00:47:33,NaN,NaN,NaN,1985,JANOWIEC WIELKOPOLSKI,NaN,None,K40,NaN,NaN,NaN,NaN
4,ADAMSKI ADAM,M,00:47:48,NaN,NaN,NaN,1986,ŻNIN,NaN,None,M30,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,ŚWIĘCICKI ANDRZEJ,M,00:50:43,NaN,NaN,NaN,1973,BYDGOSZCZ,NaN,BYDGOSKIE ŻÓŁWIE,M50,NaN,NaN,NaN,NaN
725,ŻABIŃSKI RADOSŁAW,M,NaN,NaN,00:51:21,NaN,1986,WŁOCŁAWEK,NaN,WKB MARATOŃCZYK WŁOCŁAWEK,NaN,NaN,NaN,NaN,NaN
726,ŻUCHOWSKI BARTOSZ,M,NaN,00:37:34,00:38:12,NaN,1996,PŁONNE,NaN,PŁONNE,M16,PŁONNE,None,M16,00:37:30
727,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,NaN,00:41:41,1971,RADOMIN,NaN,EKO-OPAŁ PŁONNE,M50,NaN,NaN,NaN,NaN


In [27]:
after_6[after_6['team_m'].notnull() & after_6['team_golub'].notnull() & (after_6['team_m'] != after_6['team_golub'])]

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,city_m,result_piotrkow,team_m,category_m,city_golub,team_golub,category_golub,result_golub


In [28]:
after_6[after_6['city_golub'].notnull() & after_6['city_m'].notnull() & (after_6['city_golub'] != after_6['city_m'])]

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,city_m,result_piotrkow,team_m,category_m,city_golub,team_golub,category_golub,result_golub
37,BONIECKA TERESA,K,01:14:12,01:10:23,01:11:07,01:11:31,1974,RADOMIN,01:12:00,RKS LECH RYPIN,K50,RYPIN,RKS LECH RYPIN,K50,01:11:16
78,CZERENKIEWICZ-PAŁUCKA JOANNA,K,00:50:02,00:49:15,00:49:53,00:51:07,1975,BRZEŚĆ KUJ.,00:49:51,KUNA TEAM WŁOCŁAWEK,K50,WŁOCŁAWEK,KUNA TEAM WŁOCŁAWEK,K50,00:48:17
114,DZIEŁAK ANDRZEJ,M,NaN,NaN,NaN,00:53:05,1969,LUBICZ,NaN,PARKRUN TORUŃ,NaN,LUBICZ DOLNY,PARKRUN TORUŃ,M50,00:51:31
120,DĘBSKI JAN,M,00:59:06,00:57:49,00:59:44,01:00:24,1952,DĄBROWA,01:02:55,SOŁTYS SUCHARZEWO,M70,SUCHARZEWO,SOŁTYS SUCHARZEWO,M70,00:59:06
312,KRĘC ALEKSANDER,M,00:57:31,00:55:02,00:54:26,00:55:52,1971,GNIEWKOWO,00:54:16,TG SOKÓŁ GNIEWKOWO,M50,CHRZĄSTOWO,TG SOKÓŁ GNIEWKOWO,M50,00:54:57
329,KWASIBORSKI PATRYK,M,NaN,00:35:10,00:36:17,00:36:53,1996,ROŚCISZEWO,00:36:13,GKS FABIANKI,M18,NOWE ROŚCISZEWO,GKS FABIANKI,M16,00:35:30


In [29]:
after_6[after_6['category_golub'].notnull() & after_6['category_m'].notnull() & (after_6['category_golub'] != after_6['category_m'])]

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,city_m,result_piotrkow,team_m,category_m,city_golub,team_golub,category_golub,result_golub
79,CZERWIŃSKI KRZYSZTOF,M,NaN,00:37:51,00:38:18,00:39:37,1996,CIECHOCINEK,00:38:32,WKB MARATOŃCZYK WŁOCŁAWEK,M18,CIECHOCINEK,WKB MARATOŃCZYK WŁOCŁAWEK,M16,00:37:49
329,KWASIBORSKI PATRYK,M,NaN,00:35:10,00:36:17,00:36:53,1996,ROŚCISZEWO,00:36:13,GKS FABIANKI,M18,NOWE ROŚCISZEWO,GKS FABIANKI,M16,00:35:30


In [30]:
after_6['team'] = after_6['team_golub'].combine_first(after_6['team_m'])
after_6['category'] = after_6['category_golub'].combine_first(after_6['category_m'])
after_6['city'] = after_6['city_golub'].combine_first(after_6['city_m'])
after_6 = after_6.drop(labels=['team_golub', 'team_m', 'category_golub', 'category_m', 'city_golub', 'city_m'], axis=1)

## Rypin

In [31]:
after_7 = after_6.merge(rypin, on=['name', 'gender'], how='outer', suffixes=('_m', '_rypin'))
after_7

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,result_piotrkow,result_golub,team_m,category_m,city_m,team_rypin,city_rypin,category_rypin,result_rypin
0,ABRAMOWSKI DAMIAN,M,00:37:45,NaN,NaN,NaN,1991,NaN,NaN,TT SODA,M30,ŻNIN,NaN,NaN,NaN,NaN
1,ADAMKIEWICZ SŁAWOMIR,M,NaN,NaN,NaN,NaN,<NA>,00:48:21,NaN,SGB TEAM,M50,NaN,NaN,NaN,NaN,NaN
2,ADAMSKA ANNA,K,00:57:09,NaN,NaN,NaN,1985,NaN,NaN,LECHITA BIEGA KŁECKO,K40,KŁECKO,NaN,NaN,NaN,NaN
3,ADAMSKA IZABELA,K,00:47:33,NaN,NaN,NaN,1985,NaN,NaN,None,K40,JANOWIEC WIELKOPOLSKI,NaN,NaN,NaN,NaN
4,ADAMSKI ADAM,M,00:47:48,NaN,NaN,NaN,1986,NaN,NaN,None,M30,ŻNIN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,ŻABIŃSKI RADOSŁAW,M,NaN,NaN,00:51:21,NaN,1986,NaN,NaN,WKB MARATOŃCZYK WŁOCŁAWEK,NaN,WŁOCŁAWEK,NaN,NaN,NaN,NaN
811,ŻUCHOWSKI BARTOSZ,M,NaN,00:37:34,00:38:12,NaN,1996,NaN,00:37:30,PŁONNE,M16,PŁONNE,NaN,PŁONNE,M16,00:39:16
812,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,NaN,00:41:41,1971,NaN,NaN,EKO-OPAŁ PŁONNE,M50,RADOMIN,NaN,PŁONNE,M50,00:42:17
813,ŻYTOWIECKI DAWID,M,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,RYPIN,M40,00:41:02


In [32]:
after_7[after_7['team_m'].notnull() & after_7['team_rypin'].notnull() & (after_7['team_m'] != after_7['team_rypin'])]

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,result_piotrkow,result_golub,team_m,category_m,city_m,team_rypin,city_rypin,category_rypin,result_rypin
267,KARPIŃSKI PAWEŁ,M,00:39:14,00:38:35,NaN,00:39:44,1996,NaN,00:37:38,#ZAKOLATEAM,M16,GOLUB-DOBRZYŃ,ZAKOLATEAM,GOLUB-DOBRZYŃ,M16,00:38:37
568,ROSZAK ZBIGNIEW,M,00:49:00,00:47:24,00:49:38,00:49:45,1988,00:49:15,00:48:25,WATAHA - ŁĄCZY NAS BIEGANIE,M30,GNIEWKOWO,BIEGAM DLA ZDROWIA,GNIEWKOWO,M30,00:50:05
620,STECKIEWICZ EDMUND,M,NaN,00:41:35,NaN,NaN,<NA>,NaN,NaN,RKS LECH RYPIN,M50,NaN,EDKA ZNASZ ?,RYPIN,M50,00:42:59
689,TOMASZEWSKI MACIEJ,M,NaN,00:33:46,00:33:04,00:33:15,1988,00:32:54,NaN,TASMAN TEAM,M30,TORUŃ,GKS FABIANKI,TORUŃ,M30,00:34:08


In [33]:
after_7[after_7['city_rypin'].notnull() & after_7['city_m'].notnull() & (after_7['city_rypin'] != after_7['city_m'])]

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,result_piotrkow,result_golub,team_m,category_m,city_m,team_rypin,city_rypin,category_rypin,result_rypin
43,BONIECKA TERESA,K,01:14:12,01:10:23,01:11:07,01:11:31,1974,01:12:00,01:11:16,RKS LECH RYPIN,K50,RYPIN,RKS LECH RYPIN,RADOMIN,K50,01:11:12
146,GAC MONIKA,K,00:49:27,00:47:02,00:48:12,00:48:12,1976,00:49:02,00:46:51,TG SOKÓŁ GNIEWKOWO,K40,GNIEWKOWO,TG SOKÓŁ GNIEWKOWO,SUCHATÓWKA,K40,00:52:46
215,JANOWSKI PIOTR,M,NaN,NaN,NaN,00:52:22,1964,NaN,NaN,TKKF CHEŁMNO,NaN,NOWAWIEŚ CHEŁMIŃSKA.,NaN,NOWAWIEŚ CHEŁMIŃSKA,M60,00:52:56
311,KOTUŁA PAWEŁ,M,NaN,NaN,00:45:00,00:40:28,1979,NaN,NaN,START TEAM WĄBRZEŹNO,NaN,RYŃSK,START TEAM WĄBRZEŹNO,TRZCIANO,M40,00:48:44
386,LEWANDOWSKI MACIEJ,M,00:48:30,NaN,00:48:48,00:50:37,1992,00:49:39,00:48:30,PŁONKOWO,M30,ROJEWO,NaN,PŁONKOWO,M30,00:49:55
477,OLSZEWSKI ADAM,M,NaN,00:50:25,00:53:00,00:53:29,1963,00:54:32,NaN,RKS LECH RYPIN,M60,ROGOWO,RKS LECH RYPIN,NOWY KOBRZYNIEC,M60,00:54:20
797,ŁUBECKI PATRYK,M,NaN,NaN,NaN,NaN,<NA>,NaN,00:43:30,START TEAM WĄBRZEŹNO,M30,GRUDZIĄDZ,NaN,TORUŃ,M30,00:43:09
812,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,NaN,00:41:41,1971,NaN,NaN,EKO-OPAŁ PŁONNE,M50,RADOMIN,NaN,PŁONNE,M50,00:42:17


In [34]:
after_7[after_7['category_rypin'].notnull() & after_7['category_m'].notnull() & (after_7['category_rypin'] != after_7['category_m'])]

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,result_piotrkow,result_golub,team_m,category_m,city_m,team_rypin,city_rypin,category_rypin,result_rypin
133,DŁONIAK EDMUND,M,01:02:43,01:01:45,01:04:28,01:01:51,1945,01:01:48,NaN,TG SOKÓŁ GNIEWKOWO,M80+,GNIEWKOWO,TG SOKÓŁ GNIEWKOWO,GNIEWKOWO,M80,01:01:33
516,PIETRZAK MICHAŁ,M,00:36:42,00:36:28,00:35:41,00:38:26,2005,00:37:46,NaN,KUNA TEAM WŁOCŁAWEK,M18,WŁOCŁAWEK,KUNA TEAM WŁOCŁAWEK,WŁOCŁAWEK,M16,00:36:14
609,SOWIŃSKI EDMUND,M,00:59:43,00:57:40,00:59:11,00:56:13,1942,01:00:26,NaN,TKKF PŁOCK,M80+,PŁOCK,TKKF PŁOCK,PŁOCK,M80,01:00:22
673,SŁAWIŃSKA KINGA,K,NaN,NaN,NaN,NaN,<NA>,00:52:02,NaN,GKS FABIANKI,K18,NaN,GKS FABIANKI,NOWY DUNINÓW,K16,00:50:40


In [35]:
after_7['team'] = after_7['team_rypin'].combine_first(after_7['team_m'])
after_7['category'] = after_7['category_rypin'].combine_first(after_7['category_m'])
after_7['city'] = after_7['city_rypin'].combine_first(after_7['city_m'])
after_7 = after_7.drop(labels=['team_rypin', 'team_m', 'category_rypin', 'category_m', 'city_rypin', 'city_m'], axis=1)

# Test

In [36]:
final = after_7
final.loc[final['name'] == 'TOMASZEWSKI MACIEJ', 'team'] = 'TASMAN TEAM'
final.loc[final['name'] == 'BONIECKA TERESA', 'team'] = 'RKS LECH RYPIN - niezgłoszona'
final[final['team'] == 'PARKRUN TORUŃ']

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,result_piotrkow,result_golub,result_rypin,team,category,city
11,BAJEŃSKI DARIUSZ,M,NaN,NaN,00:48:21,NaN,1993,00:49:32,00:49:15,NaN,PARKRUN TORUŃ,M30,TORUŃ
22,BARWIŃSKI LESZEK,M,NaN,00:42:45,NaN,NaN,<NA>,NaN,NaN,NaN,PARKRUN TORUŃ,M50,NaN
82,CZAJKOWSKI KAMIL,M,NaN,NaN,NaN,00:48:29,1996,NaN,00:44:46,NaN,PARKRUN TORUŃ,M16,BYDGOSZCZ
96,DEPERAS ALEKSANDER,M,NaN,00:37:30,00:38:44,00:39:44,1984,NaN,00:37:36,00:40:24,PARKRUN TORUŃ,M40,TORUŃ
126,DZIEŁAK ANDRZEJ,M,NaN,NaN,NaN,00:53:05,1969,NaN,00:51:31,NaN,PARKRUN TORUŃ,M50,LUBICZ DOLNY
131,DĘBICKI MACIEJ,M,00:42:30,NaN,00:42:35,NaN,1993,NaN,00:41:45,00:45:58,PARKRUN TORUŃ,M30,TORUŃ
141,FRĄK EDYTA,K,00:42:55,00:41:41,00:42:45,NaN,1988,00:42:57,00:42:05,NaN,PARKRUN TORUŃ,K30,TORUŃ
160,GOŁĘBIEWSKI MICHAŁ,M,NaN,00:40:41,00:42:24,00:43:51,1983,00:42:39,00:41:45,00:41:23,PARKRUN TORUŃ,M40,TORUŃ
178,GUTTFELD IGOR,M,00:39:45,00:39:24,NaN,NaN,1967,NaN,NaN,NaN,PARKRUN TORUŃ,M50,MŁYNIEC PIERWSZY
181,GÓRSKI MICHAŁ,M,NaN,00:48:25,00:52:26,00:55:47,1991,NaN,00:57:00,00:54:26,PARKRUN TORUŃ,M30,TORUŃ


In [37]:
sorted(final[final['team'].notnull()]['team'].unique())

['#SLIMAKIBARCIN',
 '#WKTM',
 '-',
 '---',
 'AGROLOK SPORT TEAM',
 'AKTYWNE LISEWO',
 'ALBO GRUBO ALBO WCALE',
 'ALL4FIT',
 'ALTOM GNIEZNO',
 'AMATEUR RUNNERS WŁOCŁAWEK',
 'ANIRO RUN TEAM',
 'ARCZCIACH',
 'BACHORZEWO',
 'BANALNY TRENER',
 'BANICI ULTRA ADVENTURE',
 'BANK SPÓŁDZIELCZY W BRODNICY',
 'BBL INOWROCŁAW',
 'BENEDEK-TEAM',
 'BIAŁE BŁOTA',
 'BIEGAM BO LUBIĘ',
 'BIEGAM DLA ZDROWIA',
 'BIEGAM SOBIE',
 'BIEGAMY LEKKĄ NÓŻKĄ',
 'BIEGOWI PRZYJACIELE',
 'BOBROWNIKI',
 'BRODNICA BIEGA',
 'BSPK BANK TEAM',
 'BWRUNNERS',
 'BYDGOSKIE ŻÓŁWIE',
 'BYDGOSZCZ',
 'BYDGOSZCZ NA START',
 'BYTOŃ',
 'BĄDKOWO',
 'CARSEKT',
 'CBR BIEGA',
 'CHOCEŃ',
 'CHOJNICE BIEGAJĄ',
 'CICHA GÓRA',
 'CYBORGI',
 'DECATHLON TEAM INOWROCŁAW',
 'DIABELSKIE PIORUNY',
 'DOBRZYŃ NAD WISŁĄ',
 'DREAM TEAM CHAWŁODNO',
 'EDKA ZNASZ ?',
 'EKO-OPAŁ PŁONNE',
 'EKONSTAL',
 'ENDO GOLUB-DOBRZYŃ',
 'ERKA RUN TEAM',
 'FABIANKI',
 'FUNDACJA SODA',
 'GDAŃSK',
 'GKS FABIANKI',
 'GLIWEK',
 'GNIEWKOWO',
 'GOLINA',
 'GP CROSS CHEŁMŻA',
 'G

## Save

In [38]:
def to_seconds(time_str):
    t = datetime.time.fromisoformat(time_str)
    return t.hour * 3600 + t.minute * 60 + t.second

def to_time_str(seconds):
    return datetime.datetime.fromtimestamp(seconds, datetime.UTC).strftime('%T')

def gp_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

def get_result(row):
    results = [row[col] for col in row.index if col.startswith('result_') and type(row[col]) == str]
    row['total_runs'] = len(results)
    best_6_runs = sorted([to_seconds(r) for r in results])[:6]
    row['avg_result'] = gp_round(sum(best_6_runs) / len(best_6_runs))
    row['avg_result_str'] = to_time_str(row['avg_result'])
    return row

In [39]:
def get_category(row):
    if pd.isnull(row['year_of_birth']):
        return row

    if row['year_of_birth'] > year - 30:
        row['category'] = row['gender'] + '18'
    elif row['year_of_birth'] > year - 40:
        row['category'] = row['gender'] + '30'
    elif row['year_of_birth'] > year - 50:
        row['category'] = row['gender'] + '40'
    elif row['year_of_birth'] > year - 60:
        row['category'] = row['gender'] + '50'
    elif row['year_of_birth'] > year - 65:
        row['category'] = row['gender'] + '60'
    elif row['year_of_birth'] > year - 70:
        row['category'] = row['gender'] + '65'
    elif row['year_of_birth'] > year - 75:
        row['category'] = row['gender'] + '70'
    elif row['year_of_birth'] > year - 80:
        row['category'] = row['gender'] + '75'
    elif row['year_of_birth'] > year - 86:
        row['category'] = row['gender'] + '80'
    else:
        row['category'] = row['gender'] + '85'

    return row
        
gp = final.apply(get_result, axis=1)
gp['year_of_birth'] = gp['year_of_birth'].astype('Int64')
gp = gp.apply(get_category, axis=1)
gp

,name,gender,result_janowiec,result_fabianki,result_radziejow,result_lisewo,year_of_birth,result_piotrkow,result_golub,result_rypin,team,category,city,total_runs,avg_result,avg_result_str
0,ABRAMOWSKI DAMIAN,M,00:37:45,NaN,NaN,NaN,1991,NaN,NaN,NaN,TT SODA,M30,ŻNIN,1,2265,00:37:45
1,ADAMKIEWICZ SŁAWOMIR,M,NaN,NaN,NaN,NaN,<NA>,00:48:21,NaN,NaN,SGB TEAM,M50,NaN,1,2901,00:48:21
2,ADAMSKA ANNA,K,00:57:09,NaN,NaN,NaN,1985,NaN,NaN,NaN,LECHITA BIEGA KŁECKO,K40,KŁECKO,1,3429,00:57:09
3,ADAMSKA IZABELA,K,00:47:33,NaN,NaN,NaN,1985,NaN,NaN,NaN,None,K40,JANOWIEC WIELKOPOLSKI,1,2853,00:47:33
4,ADAMSKI ADAM,M,00:47:48,NaN,NaN,NaN,1986,NaN,NaN,NaN,None,M30,ŻNIN,1,2868,00:47:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,ŻABIŃSKI RADOSŁAW,M,NaN,NaN,00:51:21,NaN,1986,NaN,NaN,NaN,WKB MARATOŃCZYK WŁOCŁAWEK,M30,WŁOCŁAWEK,1,3081,00:51:21
811,ŻUCHOWSKI BARTOSZ,M,NaN,00:37:34,00:38:12,NaN,1996,NaN,00:37:30,00:39:16,PŁONNE,M18,PŁONNE,4,2288,00:38:08
812,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,NaN,00:41:41,1971,NaN,NaN,00:42:17,EKO-OPAŁ PŁONNE,M50,PŁONNE,4,2482,00:41:22
813,ŻYTOWIECKI DAWID,M,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,00:41:02,NaN,M40,RYPIN,1,2462,00:41:02


In [40]:
gp.to_sql(name='results', con=cnx, if_exists='replace', index_label='id')

815